![New Relic](./assets/new-relic-logo.png)

# New Relic Playground

## Imports

Python Imports

In [ ]:
import json
import os

from textwrap import dedent

Third-party libraries

In [ ]:
import dotenv
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import requests

from requests import Session, request

## Useful definitions

In [ ]:
NULL_CURSOR = json.dumps(None)

In [ ]:
def build_query(query_string, query_params = None):
    if query_params is None:
        query_params = {}
        
    return dedent(query_string.strip()) % query_params

In [ ]:
def get_response_data(response, key_path = None, action = "actor"):
    data = response.json().get("data").get(action)

    if key_path is not None:
        for key in key_path.split(":"):
            if key.isdecimal() and isinstance(data, list):
                data = data[int(key)]
            else:
                data = data.get(key)

    return data

In [ ]:
class NewRelicGqlClient(Session):
    """Client for New Relic GraphQL API."""

    url: str = "https://api.newrelic.com/graphql"

    def __init__(self, new_relic_user_key = None):
        super().__init__()

        self.headers.update(
            {
                "Content-Type": "application/json",
                "API-Key": new_relic_user_key,
            }
        )

    def execute(self, query, variables = None, **kwargs):
        data = json.dumps(
            {
                "query": query,
                "variables": variables,
            },
        )
        return self.post(self.url, data=data, **kwargs)

## Client setup

Load credentials

In [ ]:
env_file = "../.env"

dotenv.load_dotenv(env_file)

new_relic_user_key = os.environ.get("NEW_RELIC_USER_KEY", None)

if new_relic_user_key is None:
    raise ValueError("Environment variable NEW_RELIC_USER_KEY is not set.")

Client instantiation

In [ ]:
newrelic = NewRelicGqlClient(new_relic_user_key)

Define test query

In [ ]:
query = build_query(
    """
        {
            actor {
                user {
                    email
                    name
                    id
                }
            }
        }
    """
)

Executing query

In [ ]:
response = newrelic.execute(query)

Looking for response data

In [ ]:
print(
    json.dumps(
        get_response_data(
            response, 
            action="actor",
        ),
        indent=4,
    )
)

## Playground area

### List all available dashboards

In [ ]:
query_string="""
    {
        actor {
            entitySearch(
                queryBuilder: {
                    type: DASHBOARD
                }
            ) {
                count
                results(cursor: %(cursor)s) {
                    nextCursor
                    entities {
                        ... on DashboardEntityOutline {
                            guid
                            name
                            owner {
                                email
                                userId
                            }
                            indexedAt
                            permissions
                        }
                    }
                }
            }
        }
    }
"""

query_params={
    "cursor": NULL_CURSOR,
}

query = build_query(
    query_string=query_string,
    query_params=query_params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
dashboard_count = get_response_data(response, key_path="entitySearch:count", action="actor")
dashboard_count

In [ ]:
dashboards_list = get_response_data(response, key_path="entitySearch:results:entities", action="actor")
len(dashboards_list)

In [ ]:
dashboards_list[0]

In [ ]:
next_cursor = get_response_data(response, key_path="entitySearch:results:nextCursor", action="actor")
next_cursor

### Get a single dashboard

In [ ]:
dashboard_guid = dashboards_list[0]["guid"]

In [ ]:
query_string="""
{
    actor {
        entity(guid: %(guid)s) {
            ... on DashboardEntity {
                guid
                name
                owner {
                    email
                    userId
                }
                permissions
                indexedAt
            }
        }
    }
}
"""

query_params={
    "guid": f'"{dashboard_guid}"',
}

query = build_query(
    query_string=query_string,
    query_params=query_params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
dashboard = get_response_data(response, key_path="entity", action="actor")
dashboard

### Get a dashboard with pages

In [ ]:
dashboard_guid = dashboards_list[0]["guid"]

In [ ]:
query_string="""
{
    actor {
        entity(guid: %(guid)s) {
            ... on DashboardEntity {
                guid
                name
                owner {
                    email
                    userId
                }
                permissions
                indexedAt
                pages {
                    guid
                    name
                }
            }
        }
    }
}
"""

query_params={
    "guid": f'"{dashboard_guid}"',
}

query = build_query(
    query_string=query_string,
    query_params=query_params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
dashboard = get_response_data(response, key_path="entity", action="actor")
dashboard

### Get a dashboard with pages and widget

In [ ]:
dashboard_guid = dashboards_list[0]["guid"]

In [ ]:
query_string="""
{
    actor {
        entity(guid: %(guid)s) {
            ... on DashboardEntity {
                guid
                name
                owner {
                    email
                    userId
                }
                permissions
                indexedAt
                pages {
                    guid
                    name
                    widgets {
                        id
                        rawConfiguration
                        title
                        visualization {
                            id
                        }
                    }
                }
            }
        }
    }
}
"""

query_params={
    "guid": f'"{dashboard_guid}"',
}

query = build_query(
    query_string=query_string,
    query_params=query_params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
dashboard = get_response_data(response, key_path="entity", action="actor")
dashboard.keys()

In [ ]:
dict(
    (key, value) 
    for key, value in dashboard.items() 
    if key not in ("pages")
)

In [ ]:
dashboard_pages = dashboard["pages"]
len(dashboard_pages)

In [ ]:
dashboard_page_0 = dashboard_pages[0]
dashboard_page_0.keys()

In [ ]:
dict(
    (key, value) 
    for key, value in dashboard_page_0.items() 
    if key not in ("widgets")
)

In [ ]:
dashboard_page_0_widgets = dashboard_page_0["widgets"]
len(dashboard_page_0_widgets)

In [ ]:
dashboard_page_0_widget_0 = dashboard_page_0_widgets[0]
dashboard_page_0_widget_0

### Create a dashboard

In [ ]:
query_string="""
    mutation {
        dashboardCreate(
            accountId: %(account_id)d, 
            dashboard: { 
                name: "Sample api dashboard", 
                pages: { 
                    name: "Sample api page", 
                    widgets: { 
                        configuration: { 
                            markdown: { 
                                text: "# Sample api widget"
                            }
                        }, 
                        title: ""
                    }
                }, 
                permissions: PUBLIC_READ_WRITE
            }
        ) {
            entityResult {
                name
                owner {
                    email
                    userId
                }
                guid
            }
            errors {
                description
                type
            }
        }
    }
"""

query_params={
    "account_id": ·put your account id here,
}

query = build_query(
    query_string=query_string,
    query_params=query_params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
errors = get_response_data(response, key_path="errors", action="dashboardCreate")
errors

In [ ]:
created_dashboard = get_response_data(response, key_path="entityResult", action="dashboardCreate")
created_dashboard

### Task: Create dashboard page snapshot

Read the documentation carefully and create a snapshot of an existing dashboard

In [ ]:
# Put your code here

# Define your query template
query_string="""
"""

# Define your query parameters if necessary
query_params={
}


query = build_query(
    query_string=query_string,
    query_params=query_params,
)

response = newrelic.execute(query)

In [ ]:
# only interested on the dashboard url
snapshot_url = get_response_data(response, action="dashboardCreateSnapshotUrl")
print(url_pdf)

# replace PDF with PNG, and get the link to download the file
url_png = url_pdf[:-3] + "PNG"
print(url_png)

# rename the downloaded file, and save it in the working directory
dashboard_response = requests.get(url_png, stream=True)
open('dashboard_example.png', 'wb').write(dashboard_response.content)